# Creación de los corpus para pruebas del corrector ortográfico

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re

language = "shi"

Inicialización del dataframe, eliminación de espacios blancos multiples, eliminación de espacios en blanco al inicio y final de las oraciones; y cambiar valores del dataframe vacíos a NaN

In [2]:
df = pd.read_excel("trabajo.xlsx", "Modernización", skiprows=9, usecols="B:E", names=["number", "word" , "sentence", "sentence_unnormalized"])

df = df.replace({"\s+": " "}, regex=True)
df = df.apply(lambda row: row.str.strip() if row.dtype == "object" else row)
df["sentence"] = df["sentence"].str.lower()
df["sentence_unnormalized"] = df["sentence_unnormalized"].str.lower()
df = df.replace(r"^\s*$", np.nan, regex=True)

In [3]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 1100


Remover las filas que no tengan una oración y que no tengan ningún tipo de error etiquetado

In [4]:
df = df[df["sentence"].notna()]
df = df[df["sentence_unnormalized"].notna()]

In [5]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 922


Identificar los caracteres únicos de las oraciones para propósitos de limpieza manual del excel

In [6]:
chars = sorted(list(set((df["sentence"] + df["sentence_unnormalized"]).sum())))
print("Lista de caracteres (ambos conjuntos de oraciones combinadas): {}".format(chars))

Lista de caracteres (ambos conjuntos de oraciones combinadas): [' ', "'", ',', '.', '0', '4', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '¿', 'á', 'é', 'í', 'ó']


Agregar las columnas "words" y "words_with_errors" para identificar las palabras en las oraciones que tienen errores ortográficos, también identificaremos las filas en donde las oraciones no coinciden en número de palabras

In [7]:
df["words"] = df["words_unnormalized"] = None
allowed_chars_regex = "[^0-9A-ZÁÉÍÓÚÑa-záéíóúñ\s\'\-]+"
differente_words_count_idxs = []

for _, row in df.iterrows():
    sentence = re.sub(allowed_chars_regex, '', row["sentence"])
    sentence_with_errors = re.sub(allowed_chars_regex, '', row["sentence_unnormalized"])
    
    sentence_word_list = sentence.strip().split(' ')
    sentence_with_errors_word_list = sentence_with_errors.strip().split(' ')    
    
    if(len(sentence_word_list) != len(sentence_with_errors_word_list)):
        differente_words_count_idxs.append(row["number"])
        continue
    
    errors_count = 0
    words = []
    word_errors = []
    for idx, word in enumerate(sentence_word_list):        
        if word != sentence_with_errors_word_list[idx]:
            errors_count += 1
            words.append(word)
            word_errors.append(sentence_with_errors_word_list[idx])
    
    df.loc[df["number"] == row["number"], "sentence"] = sentence
    df.loc[df["number"] == row["number"], "sentence_unnormalized"] = sentence_with_errors
    df.loc[df["number"] == row["number"], "words"] = ','.join(words)
    df.loc[df["number"] == row["number"], "words_unnormalized"] = ','.join(word_errors)

In [8]:
print("{} filas en donde el número de palabras de la oración sin errores ortográficos no es igual al número de palabras de la oracion con errores ortográficos: {}".format(len(differente_words_count_idxs), differente_words_count_idxs))

0 filas en donde el número de palabras de la oración sin errores ortográficos no es igual al número de palabras de la oracion con errores ortográficos: []


Remover las filas en donde las oraciones no coinciden en número de palabras

In [9]:
df = df[~df["number"].isin(differente_words_count_idxs)]

In [10]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 922


Agregar la columna "error_types_count" al dataframe que indique el número de "error_types"

In [11]:
df["word_unnormalized_count"] = df.apply(lambda row: len(row["words_unnormalized"].split(',')) if row["words_unnormalized"] else 0, axis=1)

Remover las filas en donde no tengan ninguna palabra con errores ortográficos.

In [12]:
df = df[~(df["word_unnormalized_count"] == 0)]

In [13]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 916


In [14]:
df.head()

,number,word,sentence,sentence_unnormalized,words,words_unnormalized,word_unnormalized_count
0,1,yamainkobo,yantashokora en meshinkonia kabori anishaman y...,yantashokora en meshinkonia cabori anishaman y...,"kabori,jawen","cabori,jahuen",2
1,2,kataxon,xawan bima ocho jemankonia chopa bená katoxon ...,xawan biman ocho jemankonia chopa bená catosho...,"bima,katoxon,nokotai","biman,catoshon,nocotai",3
2,3,tsamatabo,ochiti poiyabora tsamatabo iki jara pexe pikon...,ochiti poiyabora tsamatabo iki jara pexe pikon...,"tsinkixon,potake","tsinquixon,potaque",2
4,5,koraa,yoxan rabinra yamekiri paranta joshin koraaxon...,yoxan rabinra yamequiri paranta joshin koraaxo...,"yamekiri,bomake","yamequiri,bomaque",2
5,6,bonaketian,yametaitianra soi yakan jawen bake maxko koira...,yametaitianra soi yakan jahuen bake maxko koir...,"jawen,wetsa","jahuen,huetsa",2


In [15]:
print("Número de oraciones sin errores ortográficos y con errores ortográficos (cada uno): {}".format(df.shape[0]))

Número de oraciones sin errores ortográficos y con errores ortográficos (cada uno): 916


In [16]:
sentences = df["sentence"].tolist()
corpus = ' '.join(sentences)
words = [w.strip() for w in corpus.split()]
word_count = Counter(words)
unique_words = list(word_count.keys())

print("Número de palabras únicas (oraciones sin errores ortográficos): {}".format(len(unique_words)))

Número de palabras únicas (oraciones sin errores ortográficos): 3931


In [17]:
chars = sorted(list(set(df["sentence"].sum())))
print("{} caracteres (oraciones sin errores ortográficos): {}".format(len(chars), chars))

32 caracteres (oraciones sin errores ortográficos): [' ', "'", '0', '4', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'á', 'é', 'í', 'ó']


In [18]:
sentences = df["sentence_unnormalized"].tolist()
corpus = ' '.join(sentences)
words = [w.strip() for w in corpus.split()]
word_count = Counter(words)
unique_words = list(word_count.keys())

print("Número de palabras únicas (oraciones con errores ortográficos): {}".format(len(unique_words)))

Número de palabras únicas (oraciones con errores ortográficos): 4279


In [19]:
chars = sorted(list(set(df["sentence_unnormalized"].sum())))
print("{} caracteres (oraciones con errores ortográficos): {}".format(len(chars), chars))

33 caracteres (oraciones con errores ortográficos): [' ', "'", '0', '4', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'á', 'é', 'í', 'ó']


Crear corpus de oraciones sin errores ortograficos

In [21]:
with open(language + ".teacher_unnorm.test.sentences.txt", "w") as f:
    lines = (df["sentence"] + '\n').tolist()
    f.writelines(lines)

with open(language + ".teacher_unnorm.test.sentences.errors.txt", "w") as f:
    lines = (df["sentence_unnormalized"] + '\n').tolist()
    f.writelines(lines)